In [ ]:
# ============================================================
# Feature Selection via ElasticNet Logistic Regression
# ============================================================
# This notebook identifies the top 10 most important features
# in each of 4 scenarios using regularized logistic regression
# with ElasticNet penalty.
# ============================================================

# -------------------------
# Import Required Libraries
# -------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Plot aesthetics
sns.set_style("white")
plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 12,
    'axes.labelsize': 12,
    'axes.titlesize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})

# -------------------------
# Define Input Scenarios
# -------------------------
scenarios = {
    "Scenario_1": {
        "train": "data/df_metabol.xlsx",
        "validation": "data/combat_validation_no_scaling2.xlsx",
        "TASOAC": "data/combat_data_no_scaling2.xlsx",
        "title": "Metabolites only"
    },
    "Scenario_2": {
        "train": "data/df_ratio2.xlsx",
        "validation": "data/combat_validation_no_scaling2_ratio.xlsx",
        "TASOAC": "data/combat_data_no_scaling3_ratio.xlsx",
        "title": "Metabolites + ratios"
    },
    "Scenario_3": {
        "train": "data/df_inverse_ratios2.xlsx",
        "validation": "data/combat_validation_no_scaling2_inverse_ratio.xlsx",
        "TASOAC": "data/combat_data_no_scaling3_inverse_ratio.xlsx",
        "title": "Metabolites + inverse ratios"
    },
    "Scenario_4": {
        "train": "data/df_allratios2.xlsx",
        "validation": "data/combat_validation_no_scaling2_allratio.xlsx",
        "TASOAC": "data/combat_data_no_scaling3_all_ratios.xlsx",
        "title": "Metabolites + all ratios"
    }
}

# Threshold for progression
threshold = 0.43

# Create figure for plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
plt.subplots_adjust(wspace=0.3, hspace=1)

# Store top features from each scenario
results_dict = {}

# -------------------------
# Loop Through Scenarios
# -------------------------
for ax, (name, s) in zip(axes.flat, scenarios.items()):
    # Load datasets
    train_data = pd.read_excel(s["train"])
    validation_data = pd.read_excel(s["validation"])
    TASOAC = pd.read_excel(s["TASOAC"])

    # Clean column names
    for df in [train_data, validation_data, TASOAC]:
        df.columns = df.columns.str.replace("/", "__")

    # Use TASOAC features for Scenario 1
    if name == "Scenario_1":
        train_data = TASOAC[train_data.columns]

    # Create binary outcome
    train_data['Progressor'] = (train_data['p1'] > threshold).map({True: 'Yes', False: 'No'})
    validation_data['Progressor'] = (validation_data['p1'] > threshold).map({True: 'Yes', False: 'No'})

    # Drop unused column
    train_data = train_data.drop(columns=['p1'])
    validation_data = validation_data.drop(columns=['p1'])

    # Encode categorical variable
    if 'Sex' in train_data.columns:
        train_data = pd.get_dummies(train_data, columns=["Sex"], drop_first=True)
    if 'Sex' in validation_data.columns:
        validation_data = pd.get_dummies(validation_data, columns=["Sex"], drop_first=True)

    # Align features between train and validation
    common_cols = list(set(train_data.columns) & set(validation_data.columns))
    train_data = train_data[common_cols]

    # Separate X and y
    X = train_data.drop(columns=["Progressor"])
    y = train_data["Progressor"].map({'No': 0, 'Yes': 1})

    # Train/test split
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=123)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # ElasticNet logistic regression
    model = LogisticRegression(
        penalty="elasticnet", solver="saga", l1_ratio=0.5,
        C=1.0, class_weight="balanced", max_iter=1000, random_state=123
    )
    model.fit(X_train_scaled, y_train)

    # Get normalized coefficients
    coefs = np.abs(model.coef_[0])
    coefs = coefs / np.max(coefs) if np.max(coefs) > 0 else coefs

    # Select top 10 features
    top_idx = np.argsort(coefs)[::-1][:10]
    top_features = X.columns[top_idx]
    top_scores = coefs[top_idx]

    results_dict[s["title"]] = pd.DataFrame({
        "Feature": top_features,
        "Importance": top_scores
    })

    # Plot
    sns.barplot(
        x=top_scores, y=top_features, ax=ax,
        palette=sns.color_palette("Greys_r", 10),
        edgecolor="black", linewidth=1, width=0.4
    )
    ax.set_title(s["title"], fontweight="bold", color="black")
    ax.set_xlabel("Feature importance (abs. coefficient)", fontweight="bold", color="black")
    ax.set_ylabel("")
    ax.set_xlim(0, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# -------------------------
# Save Results
# -------------------------
plt.tight_layout()
plt.savefig("feature_importance_comparison.png", dpi=300, bbox_inches='tight')

with pd.ExcelWriter("top_10_features_all_scenarios.xlsx") as writer:
    for title, df in results_dict.items():
        df.to_excel(writer, sheet_name=title.replace(" ", "_")[:31], index=False)

print("✅ Feature selection and plot generation completed.")
